## TORONTA EXPLORER

## Introduction/buisness problem

#### A lot of potential thoughts and considerations are given to the locations whenever we visit a city. What if we could look at specific important/busy areas in the city or place of interest of a city we are visiting. In that case what if we could cluster neighbourhoods on the basis of the potential tourism intrests. It would help travellers, tourism agencies and real estate investors too.

## Explanation

#### Toronto is a fast growing finacial hub. With increasing city centres and spurling residential hubs. This makes tourism and real estate one of many growing business ventures toronto has had. The city is well connected and well planned and is financially flourishing and that aspect brings in a lot of travellers. Hence determining intrests and hotspots is definitely benificial and aids new tourists and investors

## Who are our stakeholders

#### Anyone who's intrested in the reigon, it can be individuals, tourists or even the residents who would want to know if they are in the right neibourhood acoording to thier intrests and needs. Well for tourists they get a comprehensive city guide and real estate investors get to know where thier gold lies

## What am i trying to do

#### I will use datasets regarding neighbourhoods in toronto and comlement them with foursquare API data on popular places or hotspots close to the places of interest then i can cluster the neighbourhoods with thier anemities/ hotspots and map them using folium



In [ ]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

In [2]:
from bs4 import BeautifulSoup as bs
url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text # get the html text by requests
soup = bs(url,'lxml')

In [3]:
table =soup.find_all('table', class_="wikitable sortable")
soup_cont = table[0].find_all('tr')  
print(soup_cont[1])  
clean = soup_cont[1].find_all('td') 
Postalcode = clean[0].text.strip("\n")   
Borough = clean[1].text.strip("\n")
Neighborhood = clean[2].text.strip("\n")

print('This is the first row of data :',[Postalcode,Borough,Neighborhood])

<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
This is the first row of data : ['M1A', 'Not assigned', 'Not assigned']


In [4]:
labels = {'PostalCode':[],'Borough':[],'Neighborhood':[]} 

# loop to get all of the data
for i in range(len(soup_cont)-1):  
    labels['PostalCode'].append(soup_cont[i+1].find_all('td')[0].text.strip("\n"))
    labels['Borough'].append(soup_cont[i+1].find_all('td')[1].text.strip("\n"))
    labels['Neighborhood'].append(soup_cont[i+1].find_all('td')[2].text.strip("\n"))

# dict to df
table = pd.DataFrame(data=labels) 
cols = ['PostalCode','Borough','Neighborhood']
table = table.loc[:,cols]

table.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [5]:
data = table[table.Borough != 'Not assigned']
data.loc[data.Neighborhood == 'Not assigned']
data[data.Neighborhood == 'Not assigned'].index[0] #　Find target index
data.loc[data[data.Neighborhood == 'Not assigned'].index[0],'Neighborhood'] = 'Queens Park'
data.head(5)

C:\Users\VithyabathiRanganath\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\VithyabathiRanganath\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [6]:
data = data.groupby(['PostalCode','Borough'])['Neighborhood'].apply(lambda x: "[%s]" % ', '.join(x))  
data = data.reset_index()  # 
data.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"[Rouge, Malvern]"
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]"
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]"
3,M1G,Scarborough,[Woburn]
4,M1H,Scarborough,[Cedarbrae]


In [7]:
dset=pd.read_csv('http://cocl.us/Geospatial_data')
data['Latitude']= dset['Latitude'] # adding the lattitude and longitude columns to our initial dataset
data['Longitude']= dset['Longitude']
toronto = data # named it toronto because its the city location data
toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"[Rouge, Malvern]",43.806686,-79.194353
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]",43.784535,-79.160497
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]",43.763573,-79.188711
3,M1G,Scarborough,[Woburn],43.770992,-79.216917
4,M1H,Scarborough,[Cedarbrae],43.773136,-79.239476


In [8]:
import json 
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize

!pip install folium
import folium 
print('Folium installed')
print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Folium installed
Libraries imported.


In [9]:
CLIENT_ID = 'QW3LX5SSZRNTCQAJ4TSX50MDM0WQYIZXGNHJBVUW102RQUEB' # your Foursquare ID
CLIENT_SECRET = 'DYIL1LHWBFO5GEA5BGCWYQDTKTSYIRCGU30NXTK1QL5LFOLW' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QW3LX5SSZRNTCQAJ4TSX50MDM0WQYIZXGNHJBVUW102RQUEB
CLIENT_SECRET:DYIL1LHWBFO5GEA5BGCWYQDTKTSYIRCGU30NXTK1QL5LFOLW


In [10]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [11]:
toronto_venues = getNearbyVenues(names=data['Neighborhood'],
                                   latitudes=data['Latitude'],
                                   longitudes=data['Longitude']
                                  )

[Rouge, Malvern]
[Highland Creek, Rouge Hill, Port Union]
[Guildwood, Morningside, West Hill]
[Woburn]
[Cedarbrae]
[Scarborough Village]
[East Birchmount Park, Ionview, Kennedy Park]
[Clairlea, Golden Mile, Oakridge]
[Cliffcrest, Cliffside, Scarborough Village West]
[Birch Cliff, Cliffside West]
[Dorset Park, Scarborough Town Centre, Wexford Heights]
[Maryvale, Wexford]
[Agincourt]
[Clarks Corners, Sullivan, Tam O'Shanter]
[Agincourt North, L'Amoreaux East, Milliken, Steeles East]
[L'Amoreaux West]
[Upper Rouge]
[Hillcrest Village]
[Fairview, Henry Farm, Oriole]
[Bayview Village]
[Silver Hills, York Mills]
[Newtonbrook, Willowdale]
[Willowdale South]
[York Mills West]
[Willowdale West]
[Parkwoods]
[Don Mills North]
[Flemingdon Park, Don Mills South]
[Bathurst Manor, Downsview North, Wilson Heights]
[Northwood Park, York University]
[CFB Toronto, Downsview East]
[Downsview West]
[Downsview Central]
[Downsview Northwest]
[Victoria Village]
[Woodbine Gardens, Parkview Hill]
[Woodbine Heig

In [12]:
print(toronto_venues.shape)

(1332, 7)


In [13]:
latitude = toronto.Latitude.mean()
longitude = toronto.Longitude.mean()
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(toronto_venues['Neighborhood Latitude'], toronto_venues['Neighborhood Longitude'], toronto_venues['Venue']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

In [15]:
toronto_grouped= toronto_venues.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Latitude,Venue Longitude
0,"[Adelaide, King, Richmond]",43.650571,-79.384568,43.649999,-79.384628
1,"[Agincourt North, L'Amoreaux East, Milliken, S...",43.815252,-79.284577,43.815199,-79.289821
2,[Agincourt],43.794200,-79.262029,43.793922,-79.260166
3,"[Albion Gardens, Beaumond Heights, Humbergate,...",43.739416,-79.588437,43.741636,-79.586120
4,"[Alderwood, Long Branch]",43.602414,-79.543484,43.601698,-79.545303


In [16]:
latitude = toronto.Latitude.mean()
longitude = toronto.Longitude.mean()
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(toronto_grouped['Venue Latitude'], toronto_grouped['Venue Longitude'], toronto_grouped['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

In [17]:
dset=toronto_venues.groupby('Neighborhood').count()
dset.head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"[Adelaide, King, Richmond]",30,30,30,30,30,30
"[Agincourt North, L'Amoreaux East, Milliken, Steeles East]",3,3,3,3,3,3
[Agincourt],4,4,4,4,4,4
"[Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown]",11,11,11,11,11,11
"[Alderwood, Long Branch]",10,10,10,10,10,10


In [18]:
# one hot encoding
onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot =onehot[fixed_columns]
onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
toronto_group = onehot.groupby('Neighborhood').mean().reset_index()
toronto_group.head()

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,"[Adelaide, King, Richmond]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.033333,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,"[Agincourt North, L'Amoreaux East, Milliken, S...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,[Agincourt],0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,"[Albion Gardens, Beaumond Heights, Humbergate,...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.090909,0.0,0.0,0.0,0.0,0.0
4,"[Alderwood, Long Branch]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [20]:
num_top_venues = 5

for hood in toronto_group['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_group[toronto_group['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----[Adelaide, King, Richmond]----
              venue  freq
0        Steakhouse  0.10
1              Café  0.07
2       Pizza Place  0.07
3  Asian Restaurant  0.07
4             Hotel  0.07


----[Agincourt North, L'Amoreaux East, Milliken, Steeles East]----
              venue  freq
0  Asian Restaurant  0.33
1        Playground  0.33
2              Park  0.33
3     Moving Target  0.00
4    Massage Studio  0.00


----[Agincourt]----
                venue  freq
0              Lounge  0.25
1      Breakfast Spot  0.25
2  Chinese Restaurant  0.25
3      Sandwich Place  0.25
4         Yoga Studio  0.00


----[Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown]----
                  venue  freq
0         Grocery Store  0.18
1            Beer Store  0.09
2   Fried Chicken Joint  0.09
3          Liquor Store  0.09
4  Fast Food Restaurant  0.09


----[Alderwood, Long Branch]----
          venue  freq
0   Pizza Place   0.2
1   Coffee Sh

                venue  freq
0      Baseball Field   1.0
1         Yoga Studio   0.0
2  Mac & Cheese Joint   0.0
3      Massage Studio   0.0
4      Medical Center   0.0


----[Fairview, Henry Farm, Oriole]----
            venue  freq
0  Clothing Store  0.17
1     Coffee Shop  0.10
2   Movie Theater  0.03
3    Liquor Store  0.03
4            Bank  0.03


----[First Canadian Place, Underground city]----
         venue  freq
0         Café  0.13
1  Coffee Shop  0.10
2   Restaurant  0.07
3   Steakhouse  0.07
4    Gastropub  0.07


----[Flemingdon Park, Don Mills South]----
              venue  freq
0  Asian Restaurant  0.09
1               Gym  0.09
2        Beer Store  0.09
3       Coffee Shop  0.09
4     Grocery Store  0.04


----[Forest Hill North, Forest Hill West]----
              venue  freq
0     Jewelry Store  0.25
1              Park  0.25
2  Sushi Restaurant  0.25
3             Trail  0.25
4       Yoga Studio  0.00


----[Glencairn]----
                 venue  freq
0             

                  venue  freq
0                  Park  0.11
1                   Pub  0.06
2          Burger Joint  0.06
3  Fast Food Restaurant  0.06
4         Burrito Place  0.06


----[The Beaches]----
                  venue  freq
0                 Trail   0.2
1     Health Food Store   0.2
2  Other Great Outdoors   0.2
3                   Pub   0.2
4           Yoga Studio   0.0


----[The Danforth West, Riverdale]----
                venue  freq
0    Greek Restaurant  0.27
1      Ice Cream Shop  0.07
2  Italian Restaurant  0.07
3     Bubble Tea Shop  0.03
4         Coffee Shop  0.03


----[The Junction North, Runnymede]----
               venue  freq
0      Grocery Store  0.25
1           Bus Line  0.25
2  Convenience Store  0.25
3        Pizza Place  0.25
4    Warehouse Store  0.00


----[The Kingsway, Montgomery Road, Old Mill North]----
            venue  freq
0           River  0.33
1            Park  0.33
2            Pool  0.33
3     Yoga Studio  0.00
4  Massage Studio  0.00



In [21]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [22]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_group['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_group.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"[Adelaide, King, Richmond]",Steakhouse,Asian Restaurant,Hotel,Pizza Place,Café,Lounge,Bar,Speakeasy,Coffee Shop,Seafood Restaurant
1,"[Agincourt North, L'Amoreaux East, Milliken, S...",Park,Asian Restaurant,Playground,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store,Diner,Dim Sum Restaurant
2,[Agincourt],Lounge,Chinese Restaurant,Breakfast Spot,Sandwich Place,Curling Ice,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store
3,"[Albion Gardens, Beaumond Heights, Humbergate,...",Grocery Store,Coffee Shop,Fast Food Restaurant,Beer Store,Video Store,Liquor Store,Pharmacy,Pizza Place,Sandwich Place,Fried Chicken Joint
4,"[Alderwood, Long Branch]",Pizza Place,Pool,Coffee Shop,Skating Rink,Gym,Pharmacy,Pub,Dance Studio,Sandwich Place,Women's Store


In [23]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_group.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 2, 1, 1, 1, 1, 1, 1, 1, 1])

In [24]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"[Rouge, Malvern]",43.806686,-79.194353,1.0,Fast Food Restaurant,Print Shop,Women's Store,Cupcake Shop,Dumpling Restaurant,Drugstore,Dog Run,Discount Store,Diner,Dim Sum Restaurant
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]",43.784535,-79.160497,0.0,Bar,Construction & Landscaping,Women's Store,Dance Studio,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]",43.763573,-79.188711,1.0,Breakfast Spot,Pizza Place,Tech Startup,Mexican Restaurant,Intersection,Medical Center,Rental Car Location,Electronics Store,Spa,Dim Sum Restaurant
3,M1G,Scarborough,[Woburn],43.770992,-79.216917,1.0,Coffee Shop,Korean Restaurant,Convenience Store,Curling Ice,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store,Diner
4,M1H,Scarborough,[Cedarbrae],43.773136,-79.239476,1.0,Lounge,Bank,Thai Restaurant,Bakery,Fried Chicken Joint,Caribbean Restaurant,Athletics & Sports,Hakka Restaurant,Discount Store,Diner


In [26]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
    #    color=rainbow[cluster-1],
        fill=True,
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## End